In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label
import math

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [2]:
num_qubits = 2
max_weight = 2
term_dict = {("H", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [3]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    union = used_indices_1.union(used_indices_2)
    if len(union) > 0 and len(union) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
sign_iterator = list(product([1,-1], repeat=num_qubits))
prep_string_iterator = product([math.prod([item for item in sign_tuple]) for sign_tuple in sign_iterator],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))

In [4]:
list(sign_iterator)

[(1, 1), (1, -1), (-1, 1), (-1, -1)]

In [29]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        zip(sign_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        zip(sign_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)

In [31]:
hs_experiment

[('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+XX")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+XY")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+XZ")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+YX")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+YY")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+YZ")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+ZX")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+ZY")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XX"),
  stim.PauliString("+ZZ")),
 ('h',
  stim.PauliString("+X_"),
  (1, 1),
  stim.PauliString("+XY"),
  stim.PauliString("+XX")),
 ('h',
  s

In [30]:
import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-5:-1])
            sign_string = el[-4]
            egen = ",".join(str(s) for s in el[:-5])
            coef = int(el[-1])
            ic(observable)
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

TypeError: jacobian_coefficient_calc() takes 4 positional arguments but 5 were given

In [23]:
jacobian_coef_dict["columns"]
jacobian_coef_dict["index"]

OrderedSet()

In [17]:
df

H,+X_  H,+Y_  H,+Z_  H,+_X  H,+_Y  H,+_Z  \
(1, 1)   +XX,(1, 1),+XX,+X_        0      0      0      0      0      0   
         +XX,(1, 1),+XX,+_X        0      0      0      0      0      0   
         +XX,(1, 1),+XX,+XX        0      0      0      0      0      0   
         +XX,(1, 1),+XY,+X_        0      0      0      0      0      0   
         +XX,(1, 1),+XY,+_Y        0      0      0      0      0      2   
...                              ...    ...    ...    ...    ...    ...   
(-1, -1) +YX,(-1, -1),+ZY,+_Y      0      0      0      0      0      2   
         +YX,(-1, -1),+ZY,+ZY      0      0      0      0      0      0   
         +YX,(-1, -1),+ZZ,+Z_      2      0      0      0      0      0   
         +YX,(-1, -1),+ZZ,+_Z      0      0      0      0     -2      0   
         +YX,(-1, -1),+ZZ,+ZZ      0      0      0      0      0      0   

                               H,+XX  H,+XY  H,+XZ  H,+YX  ...  A,+YY,+YZ  \
(1, 1)   +XX,(1, 1),+XX,+X_        0      0      0      0  ...         -4   
         +XX,(1, 1),+XX,+_X        0      0      0      0  ...         -4   
         +XX,(1, 1),+XX,+XX        0      0      0      0  ...          0   
         +XX,(1, 1),+XY,+X_        0      0      0      0  ...         -4   
         +XX,(1, 1),+XY,+_Y        0      0      2      0  ...          0   
...                              ...    ...    ...    ...  ...        ...   
(-1, -1) +YX,(-1, -1),+ZY,+_Y      0      0      0      0  ...          0   
         +YX,(-1, -1),+ZY,+ZY      0      2      0      0  ...          0   
         +YX,(-1, -1),+ZZ,+Z_      2      0      0      0  ...          0   
         +YX,(-1, -1),+ZZ,+_Z      0      0      0      0  ...          0   
         +YX,(-1, -1),+ZZ,+ZZ      0      0      2      0  ...          0   

                               A,+YY,+ZX  A,+YY,+ZY  A,+YY,+ZZ  A,+YZ,+ZX  \
(1, 1)   +XX,(1, 1),+XX,+X_            0         -4          0          0   
         +XX,(1, 1),+XX,+_X            0         -4          0          0   
         +XX,(1, 1),+XX,+XX            0          0          0          0   
         +XX,(1, 1),+XY,+X_            0         -4          0          0   
         +XX,(1, 1),+XY,+_Y            2          0          0          0   
...                                  ...        ...        ...        ...   
(-1, -1) +YX,(-1, -1),+ZY,+_Y          0          0          0          0   
         +YX,(-1, -1),+ZY,+ZY          0          0          0         -2   
         +YX,(-1, -1),+ZZ,+Z_          0          0          2          0   
         +YX,(-1, -1),+ZZ,+_Z          0          0          0          0   
         +YX,(-1, -1),+ZZ,+ZZ          2          0          0          0   

                               A,+YZ,+ZY  A,+YZ,+ZZ  A,+ZX,+ZY  A,+ZX,+ZZ  \
(1, 1)   +XX,(1, 1),+XX,+X_            0         -4          0          0   
         +XX,(1, 1),+XX,+_X            0         -4          0          0   
         +XX,(1, 1),+XX,+XX            0          0          0          0   
         +XX,(1, 1),+XY,+X_            0         -4          0          0   
         +XX,(1, 1),+XY,+_Y            0          0          0          4   
...                                  ...        ...        ...        ...   
(-1, -1) +YX,(-1, -1),+ZY,+_Y          0          0          0          4   
         +YX,(-1, -1),+ZY,+ZY          0          0          0          0   
         +YX,(-1, -1),+ZZ,+Z_         -2          0          0          0   
         +YX,(-1, -1),+ZZ,+_Z          0          0         -4          0   
         +YX,(-1, -1),+ZZ,+ZZ          0          0          0          0   

                               A,+ZY,+ZZ  
(1, 1)   +XX,(1, 1),+XX,+X_           -4  
         +XX,(1, 1),+XX,+_X           -4  
         +XX,(1, 1),+XX,+XX            0  
         +XX,(1, 1),+XY,+X_           -4  
         +XX,(1, 1),+XY,+_Y            0  
...                                  ...  
(-1, -1) +YX,(-1, -1),+ZY,+_Y          0  
         +Y

In [ ]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [ ]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

In [ ]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [ ]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [ ]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

In [ ]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [ ]:
df.index

In [ ]:
obs_error_rates_by_exp

In [ ]:
j = df.to_numpy()

In [ ]:
np.linalg.matrix_rank(j)

In [ ]:
jinv = np.linalg.pinv(j)

In [ ]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [ ]:
dict(zip(df.columns,intrins_errs))